In [1]:
include("../src/Utero.jl")
using .Utero


KahnTopoSort(Nodes, Edges) = Any[1, 2, 3, 4, 5]


# Differentiation for Scalar Parameters

In [2]:
Params = IdDict()
w = Utero.Tracked(7.0, Params)
grads = Utero.GetGradient(x -> 2 * (((2*w + x) - w) * w) + sin(w^2), 12.0, Params) 
# should be approx. 56.21
println(grads)

Params = IdDict()
w = Utero.Tracked(25.0, Params)
grads = Utero.GetGradient(_ -> sin(w)^2, nothing, Params) 
# should be approx. -0.26
println(grads)

Params = IdDict()
w1 = Utero.Tracked(13.2, Params); w2 = Utero.Tracked(89, Params)
grads = Utero.GetGradient(_ -> (cos(w2)^3 * 4sin(w1))^2 + cos(w1*w2)^2, nothing, Params) 
# should be approx. (27.53, 3.0428)
println(grads)


IdDict{Any, Any}(1 => 56.20829561241092)
IdDict{Any, Any}(1 => -0.26237485370392877)


IdDict{Any, Any}(2 => 3.0428042273213602, 1 => 27.53002464314667)


# Differentiation for Tensor Parameters

In [3]:

Parameters = IdDict()
w1 = Utero.Tracked(rand(2, 2), Parameters)
w2 = Utero.Tracked(rand(2, 2), Parameters)
w3 = Utero.Tracked(rand(2, 2), Parameters)

grads = Utero.GetGradient(x -> prod(*(w3, *(w2, *(w1, x)))), rand(2,1), Parameters)

IdDict{Any, Any} with 3 entries:
  2 => [0.0898584 0.0455261; 0.3949 0.200073]
  3 => [0.444011 0.336339; 0.29697 0.224955]
  1 => [0.0871254 0.193988; 0.0575244 0.12808]

The product of the parameter matrices should match A.

In [4]:
A = rand(2,2)
X = rand(100, 2)
Y = mapslices(v -> (A * v), X, dims=(2))


println(A)
println((w1 * w2 * w3).val)

for (x, y) in zip(eachrow(X), eachrow(Y))
    x = vec(x); y = vec(y)
    grads = Utero.GetGradient(x -> Utero.mse(*(w3, *(w2, *(w1, x))), y), x, Parameters)
    Utero.GradientStep(0.01, Parameters, grads)
end

println("Now, they should match")
println(A)
println((w1 * w2 * w3).val)

[0.164461776609364 0.6123083247301035; 0.893759602722423 0.07679111776713254]
[0.08070426556626836 0.18126004589428266; 0.341833789425966 0.7914596531695672]
Now, they should match


UndefVarError: UndefVarError: `mse` not defined